# 🥋 Lekcja 16: Custom Collate Fn (Obsługa danych o różnej długości)

W Inżynierii Danych PyTorch `DataLoader` działa w dwóch krokach:
1.  **Sampler** wybiera indeksy (np. `[0, 5, 2]`).
2.  **Dataset** zwraca surowe obiekty dla tych indeksów.
3.  **Collate Fn** (Sklejacz) bierze listę tych obiektów i zamienia je w jeden Tensor (Batch).

Domyślny `default_collate` robi po prostu `torch.stack()`.
Dla tekstów o różnej długości musimy napisać własny `collate_fn`, który używa **Paddingu** (wypełniania zerami).

In [1]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 1. DANE (Symulacja zdań o różnej długości)
# Wyobraź sobie, że to są ztokenizowane zdania (IDs słów).
raw_data = [
    torch.tensor([1, 2, 3]),             # Zdanie A (długość 3)
    torch.tensor([4, 5, 6, 7, 8]),       # Zdanie B (długość 5)
    torch.tensor([9]),                   # Zdanie C (długość 1)
    torch.tensor([10, 11, 12, 13])       # Zdanie D (długość 4)
]

print("--- SUROWE DANE ---")
for i, seq in enumerate(raw_data):
    print(f"Próbka {i}: {seq} (Długość: {len(seq)})")

--- SUROWE DANE ---
Próbka 0: tensor([1, 2, 3]) (Długość: 3)
Próbka 1: tensor([4, 5, 6, 7, 8]) (Długość: 5)
Próbka 2: tensor([9]) (Długość: 1)
Próbka 3: tensor([10, 11, 12, 13]) (Długość: 4)


## Problem: Domyślny Collate

Spróbujmy wrzucić to do Loadera bez żadnej konfiguracji.
Oczekujemy błędu `RuntimeError`, ponieważ PyTorch nie potrafi ułożyć "schodków" w równą macierz.

In [2]:
# batch_size=2, żeby próbował skleić przynajmniej dwa elementy
loader_broken = DataLoader(raw_data, batch_size=2, shuffle=False)

print("Próba uruchomienia domyślnego loadera...")

try:
    for batch in loader_broken:
        print(batch)
except RuntimeError as e:
    print("\n🚫 BŁĄD (Zgodnie z planem):")
    print(e)
    print("\nWyjaśnienie: stack expects each tensor to be equal size.")

Próba uruchomienia domyślnego loadera...

🚫 BŁĄD (Zgodnie z planem):
stack expects each tensor to be equal size, but got [3] at entry 0 and [5] at entry 1

Wyjaśnienie: stack expects each tensor to be equal size.


## Rozwiązanie: Padding Collate

Napiszemy funkcję, która:
1.  Przyjmuje listę tensorów (`batch`).
2.  Znajduje najdłuższy tensor.
3.  Wypełnia krótsze tensory zerami (`padding_value=0`) do tej długości.
4.  Zwraca idealny prostokąt.

Użyjemy do tego `pad_sequence` z biblioteki `torch.nn.utils.rnn`.

In [3]:
def my_padding_collate(batch):
    """
    batch: lista tensorów [tensor([1,2,3]), tensor([4,5])]
    """
    
    # 1. Padding
    # batch_first=True -> Wymiary [Batch, Time]
    # padding_value=0  -> Czym wypełniać braki? (Zazwyczaj 0 to ID dla <PAD>)
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    
    # 2. (Opcjonalnie) Zwracamy też oryginalne długości
    # To przydaje się np. w sieciach rekurencyjnych (pack_padded_sequence),
    # żeby sieć wiedziała, że te zera na końcu to śmieci.
    lengths = torch.tensor([len(x) for x in batch])
    
    return padded_batch, lengths

# Testujemy
# num_workers=0 (Dla bezpieczeństwa na Windows)
loader_fixed = DataLoader(raw_data, batch_size=2, collate_fn=my_padding_collate, shuffle=False)

print("--- WYNIK Z WŁASNYM COLLATE ---")

for i, (batch, lens) in enumerate(loader_fixed):
    print(f"\nBatch {i}:")
    print(f"Kształt: {batch.shape}")
    print(batch)
    print(f"Prawdziwe długości: {lens.tolist()}")

print("\nWidzisz zera? To jest Padding. Macierz jest prostokątna!")

--- WYNIK Z WŁASNYM COLLATE ---

Batch 0:
Kształt: torch.Size([2, 5])
tensor([[1, 2, 3, 0, 0],
        [4, 5, 6, 7, 8]])
Prawdziwe długości: [3, 5]

Batch 1:
Kształt: torch.Size([2, 4])
tensor([[ 9,  0,  0,  0],
        [10, 11, 12, 13]])
Prawdziwe długości: [1, 4]

Widzisz zera? To jest Padding. Macierz jest prostokątna!


## 🥋 Black Belt Summary

1.  **Kiedy używać?** Zawsze, gdy Twoje dane wejściowe nie są sztywną macierzą (Tekst, Audio o różnym czasie trwania, Grafy o różnej liczbie węzłów, Detekcja obiektów z różną liczbą ramek).
2.  **`pad_sequence`:** Najlepszy przyjaciel inżyniera NLP. Pamiętaj o `batch_first=True`.
3.  **Maskowanie:** W Transformerach będziesz musiał użyć tych zer, żeby stworzyć `Attention Mask` (żeby model nie zwracał uwagi na puste wypełniacze).

W następnej lekcji zajmiemy się **Samplerami**. Co zrobić, gdy masz 99% zdrowych pacjentów i 1% chorych? (Imbalanced Dataset).